In [22]:
import numpy as np

In [24]:
data = "The Dark Knight is a 2008 superhero film directed, produced, and co-written by Christopher Nolan. Featuring the DC Comics character Batman, the film is the second part of Nolan The Dark Knight Trilogy and a sequel to 2005 Batman Begins, starring an ensemble cast including Christian Bale, Michael Caine,"
chars = list(set(data))
print(chars)
data_size, vocab_size = len(data), len(chars)
# print(data_size)
# print(vocab_size)
print("data has "+str(data_size)+" characters, "+str(vocab_size)+" unique.")

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
print(char_to_ix)
print(ix_to_char)

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

['s', 't', '.', '0', 'y', 'b', ' ', 'M', 'F', 'm', '8', 'n', 'w', 'h', 'D', 'r', 'e', 'T', 'N', 'u', ',', 'p', 'q', 'K', 'B', 'k', 'C', 'i', '-', 'g', '2', 'c', '5', 'f', 'l', 'd', 'a', 'o']
data has 303 characters, 38 unique.
{'s': 0, 't': 1, '.': 2, '0': 3, 'y': 4, 'b': 5, ' ': 6, 'M': 7, 'F': 8, 'm': 9, '8': 10, 'n': 11, 'w': 12, 'h': 13, 'D': 14, 'r': 15, 'e': 16, 'T': 17, 'N': 18, 'u': 19, ',': 20, 'p': 21, 'q': 22, 'K': 23, 'B': 24, 'k': 25, 'C': 26, 'i': 27, '-': 28, 'g': 29, '2': 30, 'c': 31, '5': 32, 'f': 33, 'l': 34, 'd': 35, 'a': 36, 'o': 37}
{0: 's', 1: 't', 2: '.', 3: '0', 4: 'y', 5: 'b', 6: ' ', 7: 'M', 8: 'F', 9: 'm', 10: '8', 11: 'n', 12: 'w', 13: 'h', 14: 'D', 15: 'r', 16: 'e', 17: 'T', 18: 'N', 19: 'u', 20: ',', 21: 'p', 22: 'q', 23: 'K', 24: 'B', 25: 'k', 26: 'C', 27: 'i', 28: '-', 29: 'g', 30: '2', 31: 'c', 32: '5', 33: 'f', 34: 'l', 35: 'd', 36: 'a', 37: 'o'}


In [17]:
x = np.zeros((vocab_size,1))
print(x)

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]


In [26]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  targets.append(0)
#   print(len(inputs))
#   print(len(targets))
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
#     print("ps")
#     print(ps[t])
#     print("target")
#     print(targets[t])
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while n<=100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
  print(inputs)
  print(targets)
  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print("----\n "+txt+" \n----")

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print("iter "+str(n)+", loss: "+str(smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

[17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10]
[13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10, 6]
----
 sCytwT,e-NDCT.uMa2yiikT-lKsDcpBw.wKMfr2TfMlobBubsmleyCcqkKKaDwoFgBBkdotkCmD5NKidcde2o08 umw2rn..rT0,gsglilwCqkcTgq0-.8oqFeKf0.whMywktlmkohCKiBqCMshs02-dlu2lb0kmbgaTCqfcaTlcn2iDB.oyr hatk5D0rpubCCM02qm 
----
iter 0, loss: 90.9396496062
[6, 0, 19, 21, 16, 15, 13, 16, 15, 37, 6, 33, 27, 34, 9, 6, 35, 27, 15, 16, 31, 1, 16, 35, 20]
[0, 19, 21, 16, 15, 13, 16, 15, 37, 6, 33, 27, 34, 9, 6, 35, 27, 15, 16, 31, 1, 16, 35, 20, 6]
[6, 21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11]
[21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11, 6]
[6, 5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 18, 37, 34, 36, 11, 2, 6, 8, 16]
[5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 18, 37, 34, 36, 11, 2, 6, 8, 16, 36]
[36, 1, 19,

[5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 18, 37, 34, 36, 11, 2, 6, 8, 16, 36]
[36, 1, 19, 15, 27, 11, 29, 6, 1, 13, 16, 6, 14, 26, 6, 26, 37, 9, 27, 31, 0, 6, 31, 13, 36]
[1, 19, 15, 27, 11, 29, 6, 1, 13, 16, 6, 14, 26, 6, 26, 37, 9, 27, 31, 0, 6, 31, 13, 36, 15]
[15, 36, 31, 1, 16, 15, 6, 24, 36, 1, 9, 36, 11, 20, 6, 1, 13, 16, 6, 33, 27, 34, 9, 6, 27]
[36, 31, 1, 16, 15, 6, 24, 36, 1, 9, 36, 11, 20, 6, 1, 13, 16, 6, 33, 27, 34, 9, 6, 27, 0]
[0, 6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36]
[6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36, 11]
[11, 6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4]
[6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4, 6]
[6, 36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6, 24, 36, 1]
[36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6,

[0, 19, 21, 16, 15, 13, 16, 15, 37, 6, 33, 27, 34, 9, 6, 35, 27, 15, 16, 31, 1, 16, 35, 20, 6]
[6, 21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11]
[21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11, 6]
[6, 5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 18, 37, 34, 36, 11, 2, 6, 8, 16]
[5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 18, 37, 34, 36, 11, 2, 6, 8, 16, 36]
[36, 1, 19, 15, 27, 11, 29, 6, 1, 13, 16, 6, 14, 26, 6, 26, 37, 9, 27, 31, 0, 6, 31, 13, 36]
[1, 19, 15, 27, 11, 29, 6, 1, 13, 16, 6, 14, 26, 6, 26, 37, 9, 27, 31, 0, 6, 31, 13, 36, 15]
----
 sc ttiyaas2asT-y. ,hatekFts..aseosgas.dy ,  dsdirrndheoCBe00Ft w an aderr aothcuope b,adtaidDp aonho0usctFBic tn iy bFd0FcanD dKlqucw20smaFihwlCy io e,an- i0 elt r aFe0o ao a hDylasTue.cyraFgCohtFhaF  
----
iter 100, loss: 91.8082681068
[15, 36, 31, 1, 16, 15, 6, 24, 36, 1, 9, 36, 11, 20, 6, 1, 13, 16, 6, 33, 27, 34, 9, 6, 27]
[36, 31, 

[0, 6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36]
[6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36, 11]
[11, 6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4]
[6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4, 6]
[6, 36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6, 24, 36, 1]
[36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6, 24, 36, 1, 9]
[9, 36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16]
[36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16, 11]
[11, 0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13]
[0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13, 15]
[15, 27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 1

----
 aaoNndenqgeienlrhy a  hask uC rht o focght2Dora ay sn,tre5hlr ccninMrp rol  na8e tn  yiak KBtrlemniThoChr i 0hiKptThr 2tgk 8hrmha DlgrtC5 tr KllrTo0Mtdndgh Nrogri.  gyorplunt ld roghert Nnisccaaismi2i 
----
iter 200, loss: 90.1138009964
[9, 36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16]
[36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16, 11]
[11, 0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13]
[0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13, 15]
[15, 27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27]
[27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27, 11]
[17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10]
[13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10, 6]
[6, 0, 19, 

...

[11, 0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13]
[0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13, 15]
[15, 27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27]
[27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27, 11]
[17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10]
[13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10, 6]
[6, 0, 19, 21, 16, 15, 13, 16, 15, 37, 6, 33, 27, 34, 9, 6, 35, 27, 15, 16, 31, 1, 16, 35, 20]
[0, 19, 21, 16, 15, 13, 16, 15, 37, 6, 33, 27, 34, 9, 6, 35, 27, 15, 16, 31, 1, 16, 35, 20, 6]
[6, 21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11]
[21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11, 6]
[6, 5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 1

[6, 21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11]
[21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37, 28, 12, 15, 27, 1, 1, 16, 11, 6]
[6, 5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 18, 37, 34, 36, 11, 2, 6, 8, 16]
[5, 4, 6, 26, 13, 15, 27, 0, 1, 37, 21, 13, 16, 15, 6, 18, 37, 34, 36, 11, 2, 6, 8, 16, 36]
[36, 1, 19, 15, 27, 11, 29, 6, 1, 13, 16, 6, 14, 26, 6, 26, 37, 9, 27, 31, 0, 6, 31, 13, 36]
[1, 19, 15, 27, 11, 29, 6, 1, 13, 16, 6, 14, 26, 6, 26, 37, 9, 27, 31, 0, 6, 31, 13, 36, 15]
[15, 36, 31, 1, 16, 15, 6, 24, 36, 1, 9, 36, 11, 20, 6, 1, 13, 16, 6, 33, 27, 34, 9, 6, 27]
[36, 31, 1, 16, 15, 6, 24, 36, 1, 9, 36, 11, 20, 6, 1, 13, 16, 6, 33, 27, 34, 9, 6, 27, 0]
[0, 6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36]
[6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36, 11]
[11, 6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17,

[11, 6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4]
[6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4, 6]
[6, 36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6, 24, 36, 1]
[36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6, 24, 36, 1, 9]
[9, 36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16]
[36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16, 11]
[11, 0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13]
[0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13, 15]
[15, 27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27]
[27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27, 11]
[17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 

[9, 36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16]
[36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16, 11]
[11, 0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13]
[0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11, 29, 6, 26, 13, 15]
[15, 27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27]
[27, 0, 1, 27, 36, 11, 6, 24, 36, 34, 16, 20, 6, 7, 27, 31, 13, 36, 16, 34, 6, 26, 36, 27, 11]
[17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10]
[13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 27, 0, 6, 36, 6, 30, 3, 3, 10, 6]
[6, 0, 19, 21, 16, 15, 13, 16, 15, 37, 6, 33, 27, 34, 9, 6, 35, 27, 15, 16, 31, 1, 16, 35, 20]
[0, 19, 21, 16, 15, 13, 16, 15, 37, 6, 33, 27, 34, 9, 6, 35, 27, 15, 16, 31, 1, 16, 35, 20, 6]
[6, 21, 15, 37, 35, 19, 31, 16, 35, 20, 6, 36, 11, 35, 6, 31, 37

[15, 36, 31, 1, 16, 15, 6, 24, 36, 1, 9, 36, 11, 20, 6, 1, 13, 16, 6, 33, 27, 34, 9, 6, 27]
[36, 31, 1, 16, 15, 6, 24, 36, 1, 9, 36, 11, 20, 6, 1, 13, 16, 6, 33, 27, 34, 9, 6, 27, 0]
[0, 6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36]
[6, 1, 13, 16, 6, 0, 16, 31, 37, 11, 35, 6, 21, 36, 15, 1, 6, 37, 33, 6, 18, 37, 34, 36, 11]
[11, 6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4]
[6, 17, 13, 16, 6, 14, 36, 15, 25, 6, 23, 11, 27, 29, 13, 1, 6, 17, 15, 27, 34, 37, 29, 4, 6]
[6, 36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6, 24, 36, 1]
[36, 11, 35, 6, 36, 6, 0, 16, 22, 19, 16, 34, 6, 1, 37, 6, 30, 3, 3, 32, 6, 24, 36, 1, 9]
[9, 36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16]
[36, 11, 6, 24, 16, 29, 27, 11, 0, 20, 6, 0, 1, 36, 15, 15, 27, 11, 29, 6, 36, 11, 6, 16, 11]
[11, 0, 16, 9, 5, 34, 16, 6, 31, 36, 0, 1, 6, 27, 11, 31, 34, 19, 35, 27, 11

In [36]:
def predict(inputs,hprev):
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  predicted=-1
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t])
    predicted = np.argmax(ps[t])
  print(ix_to_char[predicted])

In [43]:
input = [24,36,1,9,36,11] ##Batman ##[18,37,34,36] ##Nola
hprev = np.zeros((hidden_size,1))
predict(input,hprev)

,
